# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298402375776                   -0.85    5.1         
  2   -8.300195722552       -2.75       -1.26    1.1    108ms
  3   -8.300431150090       -3.63       -1.89    3.0    192ms
  4   -8.300462051261       -4.51       -2.77    1.5    122ms
  5   -8.300464236421       -5.66       -3.06    4.2    178ms
  6   -8.300464441812       -6.69       -3.23    6.0    207ms
  7   -8.300464542665       -7.00       -3.38    2.0    157ms
  8   -8.300464598441       -7.25       -3.54    1.1    170ms
  9   -8.300464630975       -7.49       -3.74    1.6    146ms
 10   -8.300464635746       -8.32       -3.83    1.0    118ms
 11   -8.300464642517       -8.17       -4.07    1.6    142ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63725296025                   -0.71    6.5         
  2   -16.67712755773       -1.40       -1.13    1.4    285ms
  3   -16.67918237114       -2.69       -1.87    2.9    371ms
  4   -16.67927585989       -4.03       -2.59    2.8    383ms
  5   -16.67928581768       -5.00       -3.11    4.9    465ms
  6   -16.67928618170       -6.44       -3.49    3.1    368ms
  7   -16.67928621673       -7.46       -4.11    1.8    300ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32514717426                   -0.56    7.1         
  2   -33.33260576824       -2.13       -1.00    1.0    1.10s
  3   -33.33580425094       -2.50       -1.70    3.2    1.36s
  4   -33.33682448396       -2.99       -2.28    8.0    1.82s
  5   -33.33692554376       -4.00       -2.70    7.1    1.74s
  6   -33.33694353360       -4.74       -3.62    3.0    1.40s
  7   -33.33694372505       -6.72       -3.89    4.6    1.70s
  8   -33.33694378235       -7.24       -4.90    2.2    1.18s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298207335062                   -0.85    5.0         
  2   -8.300226033481       -2.69       -1.58    1.0   89.7ms
  3   -8.300378472016       -3.82       -2.32    3.2    156ms
  4   -8.300321809512   +   -4.25       -2.17    2.6    135ms
  5   -8.300463290929       -3.85       -3.30    1.1   93.0ms
  6   -8.300464528866       -5.91       -3.68    3.0    150ms
  7   -8.300464635351       -6.97       -4.14    1.4    101ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32556846557                   -0.56    7.0         
  2   -33.29396188942   +   -1.50       -1.26    1.1    923ms
  3   +4.189523656922   +    1.57       -0.28    6.5    2.19s
  4   -33.32986175263        1.57       -1.84    6.5    2.03s
  5   -33.24151071633   +   -1.05       -1.35    4.0    1.73s
  6   -33.26224403885       -1.68       -1.60    3.5    1.48s
  7   -32.98959474714   +   -0.56       -1.29    5.1    1.60s
  8   -33.33442372383       -0.46       -2.20    4.5    1.45s
  9   -33.33583271146       -2.85       -2.36    2.1    1.15s
 10   -33.33681472816       -3.01       -2.69    2.2    1.18s
 11   -33.33691635406       -3.99       -3.08    2.1    1.12s
 12   -33.33693190733       -4.81      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.